In [1]:
%env OMP_NUM_THREADS=1
%env MKL_NUM_THREADS=1

import aqlm
import torch
torch.set_num_threads(1)
from torch import nn
from torch.nn import functional as F

env: OMP_NUM_THREADS=1
env: MKL_NUM_THREADS=1


In [2]:
num_inputs = 1
out_features = 8192
in_features = 8192
out_group_size = 1
num_out_groups = out_features // out_group_size
in_group_size = 32
num_in_groups = in_features // in_group_size
num_codebooks = 8
nbits_per_codebook = 8
codebook_size = 2**nbits_per_codebook

device = 'cpu'
dtype = torch.float32

input = torch.rand((num_inputs, in_features), dtype=dtype, device=device)
codes = aqlm.utils.pack_int_data(
    torch.randint(0, 2**nbits_per_codebook, size=(num_out_groups, num_in_groups, num_codebooks), dtype=torch.int32, device=device),
    nbits_per_codebook,
)

codebooks = torch.rand((num_codebooks, codebook_size, out_group_size, in_group_size), dtype=dtype, device=device).normal_(1, 1)
scales = torch.rand((num_out_groups, 1, 1, 1), device=device, dtype=dtype)

weight = aqlm.utils._dequantize_weight(aqlm.utils.unpack_int_data(codes, nbits_per_codebook), codebooks, scales).contiguous()
bias = None # torch.rand((out_features,), dtype=dtype, device=device)

In [3]:
codes = torch.permute(codes, (1, 0, 2)).contiguous()

In [4]:
for _ in range(10):
    _ = aqlm.inference_kernels.kernel_selector.forward_pass_quantized_linear(input, codes, codebooks, scales, bias)
# torch.cuda.synchronize()

Compiling kernel_key=(32, 8192, 8192, 8)


/home/blacksamorez/AQLM/.conda/lib/python3.11/site-packages/numba/core/decorators.py:282: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [5]:
%%time
for i in range(100):
    # torch.cuda.synchronize()
    _ = aqlm.inference_kernels.kernel_selector.forward_pass_quantized_linear(input, codes, codebooks, scales, bias)
# torch.cuda.synchronize()

CPU times: user 864 ms, sys: 404 µs, total: 865 ms
Wall time: 864 ms


In [6]:
for _ in range(10):
    _ = F.linear(input, weight, bias)
# torch.cuda.synchronize()

In [7]:
%%time
for i in range(100):
    # torch.cuda.synchronize()
    _ = F.linear(input, weight, bias)
# torch.cuda.synchronize()

CPU times: user 1.86 s, sys: 372 µs, total: 1.86 s
Wall time: 1.86 s
